In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install -q pyspark

import os

os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark import SparkFiles
from pyspark.sql.types import *


spark = SparkSession.builder.appName("Processa CSV").getOrCreate()


spark.sparkContext.addFile("https://noverde-data-engineering-test.s3.amazonaws.com/loans_sample.csv")
df = spark.read.csv(SparkFiles.get("loans_sample.csv"), header=True)


spark.sparkContext.addFile("https://noverde-data-engineering-test.s3.amazonaws.com/payments_sample.parquet")
df1 = spark.read.parquet(SparkFiles.get("payments_sample.parquet"))

spark.sparkContext.addFile("https://noverde-data-engineering-test.s3.amazonaws.com/installments_sample.json")
rdd = spark.read.json(SparkFiles.get("installments_sample.json"))


joinedDF = df1.join(df, df.loan_id == df1.loan_id).drop(df1.loan_id)

final_df=joinedDF.select(joinedDF["loan_id"],joinedDF["period"],joinedDF["accepted_at"], joinedDF["payday"], joinedDF["interest_rate"])


final_df.write.parquet("/temp/")

AnalysisException: ignored